In [2]:
!pip install transformers

     |████████████████████████████████| 2.1MB 10.0MB/s 
     |████████████████████████████████| 3.3MB 55.3MB/s 
     |████████████████████████████████| 901kB 53.2MB/s 


In [9]:
from tokenizers import SentencePieceBPETokenizer
from tokenizers.normalizers import BertNormalizer

train_data = '/content/drive/MyDrive/History_SentenceMaker/history_data/history_sentence.txt'

tokenizer = SentencePieceBPETokenizer()

tokenizer._tokenizer.normalizer = BertNormalizer(clean_text=True,
handle_chinese_chars=False,
lowercase=False)

tokenizer.train(
    train_data,
    vocab_size=1000,
    special_tokens=["<s>", "<pad>", "</s>", "<unk>",]
)

tokenizer.save_model("/content/drive/MyDrive/History_SentenceMaker/model")
tokenizer = SentencePieceBPETokenizer.from_file(vocab_filename="/content/drive/MyDrive/History_SentenceMaker/model/vocab.json", merges_filename="/content/drive/MyDrive/History_SentenceMaker/model/merges.txt")

tokenizer.add_special_tokens(["<s>", "</s>", "<unk>", "<pad>", "<shkim>"])
tokenizer.pad_token_id = tokenizer.token_to_id("<pad>")
tokenizer.unk_token_id = tokenizer.token_to_id("<unk>")
tokenizer.bos_token_id = tokenizer.token_to_id("<bos>")
tokenizer.eos_token_id = tokenizer.token_to_id("<eos>")

In [10]:
from transformers import GPT2Config, GPT2LMHeadModel
# creating the configurations from which the model can be made
config = GPT2Config(
  vocab_size=tokenizer.get_vocab_size(),
  bos_token_id=tokenizer.token_to_id("<s>"),
  eos_token_id=tokenizer.token_to_id("</s>"),
)
# creating the model
model = GPT2LMHeadModel(config)

In [11]:
import json
import os
import pickle
import random
import time
import warnings
from typing import Dict, List, Optional

import torch
from torch.utils.data.dataset import Dataset

from filelock import FileLock

from transformers.tokenization_utils import PreTrainedTokenizer
from transformers.utils import logging
from transformers import DataCollatorForLanguageModeling

logger = logging.get_logger(__name__)

class TextDataset(Dataset):
    """
    This will be superseded by a framework-agnostic approach soon.
    """

    def __init__(
        self,
        tokenizer: PreTrainedTokenizer,
        file_path: str,
        block_size: int,
        overwrite_cache=False,
        cache_dir: Optional[str] = None,
    ):
        assert os.path.isfile(file_path), f"Input file path {file_path} not found"

        block_size = block_size - tokenizer.num_special_tokens_to_add(is_pair=False)

        directory, filename = os.path.split(file_path)
        cached_features_file = os.path.join(
            cache_dir if cache_dir is not None else directory,
            "cached_lm_{}_{}_{}".format(
                tokenizer.__class__.__name__,
                str(block_size),
                filename,
            ),
        )

        # Make sure only the first process in distributed training processes the dataset,
        # and the others will use the cache.
        lock_path = cached_features_file + ".lock"
        with FileLock(lock_path):

            if os.path.exists(cached_features_file) and not overwrite_cache:
                start = time.time()
                with open(cached_features_file, "rb") as handle:
                    self.examples = pickle.load(handle)
                logger.info(
                    f"Loading features from cached file {cached_features_file} [took %.3f s]", time.time() - start
                )

            else:
                logger.info(f"Creating features from dataset file at {directory}")
                self.examples = []
                text = ""
                with open(file_path, encoding="utf-8") as f:
                    lines = f.readlines()
                    for line in lines:
                        line = line.strip()
                        line = "<s>"+line+"</s>" # 학습 데이터 앞 뒤에 문장 구분 기호를 추가
                        text += line    # 'text' 객체에 모든 학습 데이터를 다 합친다
                tokenized_text = tokenizer.encode(text).ids

                # 모델의 최대 sequence length만큼 데이터를 잘라서 저장
                for i in range(0, len(tokenized_text) - block_size + 1, block_size):  # Truncate in block of block_size
                    self.examples.append(
                        tokenized_text[i : i + block_size]
                    )
                # Note that we are losing the last truncated example here for the sake of simplicity (no padding)
                # If your dataset is small, first you should look for a bigger one :-) and second you
                # can change this behavior by adding (model specific) padding.

                start = time.time()
                with open(cached_features_file, "wb") as handle:
                    pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)
                logger.info(
                    "Saving features into cached file %s [took %.3f s]", cached_features_file, time.time() - start
                )

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i) -> torch.Tensor:
        return torch.tensor(self.examples[i], dtype=torch.long)

dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=train_data,
    block_size=32,
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,# GPT는 생성모델이기 때문에 [MASK] 가 필요 없음
)

In [30]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='model_output',
    overwrite_output_dir=True,
    num_train_epochs=50,
    per_device_train_batch_size=32,
    save_steps=100,
    save_total_limit=5,
    logging_steps=100

)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

trainer.train()
trainer.save_model()

Step,Training Loss
100,0.137200
200,0.122400
300,0.114000
400,0.109100
500,0.100900
600,0.094900
700,0.090800
800,0.087400
900,0.084100
1000,0.079600


In [31]:
import torch

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

input_ids = torch.tensor(tokenizer.encode("<s>", add_special_tokens=True).ids).unsqueeze(0).to('cuda')

output_sequences = model.generate(input_ids=input_ids, do_sample=True, max_length=100, num_return_sequences=10)
print()

for generated_sequence in output_sequences:
    generated_sequence = generated_sequence.tolist()
    print(tokenizer.decode(generated_sequence, skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



세종 재위 시기에는 우리 풍토에 맞는 농법을 소개한 『농사직설』이 간행되었다.
청·프 저쟁이 벌어진 틈을 타 정변을 일으켰다는 것을 통해 갑신정변임을 알 수 있다.
문벌귀족은 정치적 특권으로 음서제를 문벌귀족은 경제적 특권으로 공음전을 묘청 특권범전을 음서론화 받제를 문묘청천벌전을 묘청사관과 대화 공음전을 경제변청의 경제청 때 사청을 묘청 4에서는 벌화경화 부어사의 시행하였다.현산서 경제적 정묘과 후흥통령
공민왕은 정동행성 이문소를 폐지하였다.
신라 성덕왕은 백성에게 정전을 지급하였다.
순종의 인산일을 기회로 전개하였다.
대한민국 임시 정부가 조소앙의 삼균주의에 바탕을 둔 건국 강령을 발표한 것은 194 『아 보탐앙의 지계한 것은 남령왕 단하으며, 삼획 기무원용 단독립의 군사동 최주 강식, 미령 유일 대원군 나합령을 줄재금지방을 발표산성적주의와 지청제복무 국안을 지
성왕은 수도를 사비로 천도하고 국호를 남부여로 고쳤다.
신라 성덕왕은 백성에게 정전을 지급하였다.
선농단은 조선 시대와 중국에서 농사와 인연이 깊은 신농씨와 후직씨를 주신으로 모씨 무씨직씨를 저술하였법 왕 과관을판조과 주신으로 모계국 발해계왕된 사건이다.
